In [ ]:
import pandas as pd         # 导入pandas库，它就像一个超级强大的Excel工具，专门用来处理表格数据（DataFrame）。
from sklearn.model_selection import train_test_split # 从sklearn.model_selection模块导入train_test_split函数，用于将数据集划分为训练集和测试集。
from sklearn.linear_model import LogisticRegression # 从sklearn.linear_model模块导入LogisticRegression类，这是我们将要使用的逻辑回归模型。
import pickle               # 导入pickle库，用于序列化和反序列化Python对象，这里主要用于保存和加载训练好的模型。
from sklearn.metrics import classification_report # 从sklearn.metrics模块导入classification_report函数，用于生成详细的分类性能报告。
from imblearn.over_sampling import SMOTE # 从imblearn.over_sampling模块导入SMOTE类，用于处理数据类别不平衡问题（过采样）。

# 加载数据
data = pd.read_csv('finance数据集.csv') # 使用pandas的read_csv函数，读取名为 'finance数据集.csv' 的CSV文件到DataFrame `data` 中。

# 显示前五行的数据
print(data.head())          # 打印DataFrame `data` 的前5行，帮助用户快速预览数据。

# 选择自变量和因变量
X = data.drop(['SeriousDlqin2yrs', 'Unnamed: 0'], axis=1)
# 从DataFrame `data` 中删除 'SeriousDlqin2yrs'（目标变量）和 'Unnamed: 0'（通常是无用索引列）。
# `axis=1` 表示按列删除。剩余的列作为特征（自变量），赋值给 `X`。
y = data['SeriousDlqin2yrs'] # 选取 'SeriousDlqin2yrs' 列作为目标变量（因变量），赋值给 `y`。

# 分割训练集和测试集（测试集20%）
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
# 使用train_test_split函数将数据集划分为训练集和测试集。
# `X`, `y`: 要分割的特征和目标变量。
# `test_size=0.2`: 测试集占总数据的20%。
# `random_state=42`: 随机种子，确保每次运行代码时分割结果一致。

# 训练Logistic回归模型（最大迭代次数为1000次）
model = LogisticRegression(max_iter=1000)
# 创建一个LogisticRegression模型实例。
# `max_iter=1000`: 设置最大迭代次数为1000，以确保模型有足够的时间收敛。
#训练 Logistic 回归模型
model.fit(X_train,y_train)  # 使用训练集特征 `X_train` 和训练集目标 `y_train` 训练Logistic回归模型。

# 保存模型
with open('2.2.1_model.pkl', 'wb') as file: # 以二进制写入模式 ('wb') 打开一个文件，用于保存模型。
    pickle.dump(model,file) # 使用pickle.dump() 将 `model` 对象序列化并写入文件。

# 预测并保存结果
y_pred = model.predict(X_test) # 使用训练好的 `model` 对测试集特征 `X_test` 进行类别预测。
pd.DataFrame(y_pred, columns=['预测结果']).to_csv('2.2.1_results.txt', index=False)
# 将预测结果 `y_pred` 转换为一个pandas DataFrame，列名为 '预测结果'。
# 然后使用 `to_csv()` 方法将其保存为CSV文件 `2.2.1_results.txt`。
# `index=False` 表示在保存时不要写入DataFrame的索引。

# 生成测试报告
report = classification_report(y_test, y_pred, zero_division=1)
# 使用 `classification_report` 函数生成详细的分类性能报告。
# `y_test`: 真实的测试集目标值。
# `y_pred`: 模型在测试集上的预测结果。
# `zero_division=1`: 如果某个标签没有任何预测样本，则其精确度、召回率、F1分数将设为1。
with open('2.2.1_report.txt', 'w') as file: # 以写入模式 ('w') 打开一个文本文件，用于保存报告。
    file.write(report)                  # 将分类报告内容写入文件。

# 分析测试结果
accuracy = (y_pred==y_test).mean()  # 计算准确率：比较预测结果 `y_pred` 和真实结果 `y_test`，
                                # `==` 会生成一个布尔Series，`mean()` 计算True的比例（即准确率）。
print(f"模型准确率: {accuracy:.2f}") # 打印模型的准确率，保留两位小数。

# 处理数据不平衡
smote = SMOTE(random_state=42)      # 创建一个SMOTE实例，用于过采样少数类别样本。
                                    # `random_state=42` 确保生成合成样本的可复现性。
X_resampled, y_resampled = smote.fit_resample(X_train,y_train)
# 对训练集 `X_train` 和 `y_train` 应用SMOTE进行过采样。
# SMOTE会基于少数类别样本生成新的合成样本，使类别分布更加平衡。
# `X_resampled` 和 `y_resampled` 是过采样后的新的训练集。

# 重新训练模型
model.fit(X_resampled, y_resampled) # 使用经过SMOTE处理后的训练集 `X_resampled` 和 `y_resampled` 重新训练模型。
# 重新预测
y_pred_resampled = model.predict(X_test) # 使用SMOTE处理后训练的模型对测试集 `X_test` 重新进行类别预测。

# 保存新结果
pd.DataFrame(y_pred_resampled, columns=['预测结果']).to_csv('2.2.1_results_xg.txt', index=False)
# 将重新预测的结果 `y_pred_resampled` 保存为CSV文件 `2.2.1_results_xg.txt`。

# 生成新的测试报告
report_resampled = classification_report(y_test, y_pred_resampled, zero_division=1)
# 使用 `classification_report` 函数生成SMOTE处理后模型的分类性能报告。
with open('2.2.1_report_xg.txt', 'w') as file: # 打开文件保存报告。
    file.write(report_resampled)            # 将报告内容写入文件。

# 分析新的测试结果
accuracy_resampled = (y_test == y_pred_resampled).mean()
# 计算SMOTE处理后模型的准确率。
print(f"重新采样后的模型准确率: {accuracy_resampled:.2f}") # 打印SMOTE处理后模型的准确率。
